In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import stats, sem
from scipy.stats import f_oneway, pearsonr
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.power import NormalIndPower, TTestIndPower
from statsmodels.stats.weightstats import ztest, ttest_ind
from scipy.stats import chi2_contingency, shapiro, t


In [ ]:
#montagem do drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url_test = '/content/drive/MyDrive/Colab Notebooks/teste_de_hipotese/experimento_teste_ab.csv'
url_pacient = '/content/drive/MyDrive/Colab Notebooks/teste_de_hipotese/pacientes.csv'

In [ ]:
df_test = pd.read_csv(url_test)
df_pacient = pd.read_csv(url_pacient)

In [ ]:
df_test.head()

,Visitante_ID,Versão_Página,Visualizações,Conversões
0,1,A,167,1
1,2,A,160,0
2,3,A,170,0
3,4,A,161,1
4,5,A,104,0


Considerando a base de dados experimento_test_ab.csv responda as
questões abaixo:
1. Qual dos cenários tem a maior taxa de conversão?


In [ ]:
#calculando a taxa de conversão para cada grupo
conversion_stats = df_test.groupby('Versão_Página').agg(
    total_visitantes = ('Visitante_ID','count'),
    total_conversões = ('Conversões','sum')
)
conversion_stats['Taxa_conversão'] = conversion_stats['total_conversões'] / conversion_stats['total_visitantes']
print('Taxa de conversão para cada grupo:')
print(conversion_stats)

Taxa de conversão para cada grupo:
               total_visitantes  total_conversões  Taxa_conversão
Versão_Página                                                    
A                          1000               105           0.105
B                          1000               125           0.125


2. Calcule qual o tamanho da amostra necessária para o desenvolvimento
de um teste A/B, seguindo os seguintes critérios:
* a. O cenário A, da base, como o inicial, que funciona hoje.
* b. Considere 95% de confiança de que o efeito na conversão não foi aleatório com um nível de signicância de 5% (alpha = 0,05).
* c. Também considere 80% de certeza conseguir capturar o efeito
da nova abordagem.
* d. O aumento para 10% de conversão


In [ ]:
# Parâmetros
p1 = 0.105
p2 = 0.12
alpha = 0.05
power = 0.8

# Z-scores
z_alpha = 1.96  # para alpha = 0.05 (95% confiança)
z_beta = 0.84   # para power = 0.8 (80% poder)

# Média das proporções
p_bar = (p1 + p2) / 2

# Cálculo da amostra por grupo
numerador = 2 * (z_alpha + z_beta)**2 * p_bar * (1 - p_bar)
denominador = (p1 - p2)**2

n = numerador / denominador
print(f"Tamanho de amostra necessário por grupo: {round(n)}")

Tamanho de amostra necessário por grupo: 6958


In [ ]:
df_pacient.head()

,Unnamed: 0,Pressao_Arterial,Idade,Genero,Etnia,Estado_Saude,Nome_Genero,Nome_Etnia,Nome_Estado_Saude
0,0,142.755809,31,1,1,1,Feminino,Afro-americano,Com condições de saúde adicionais
1,1,161.878468,35,1,1,0,Feminino,Afro-americano,Saudável
2,2,161.508259,39,0,3,1,Masculino,Asiático,Com condições de saúde adicionais
3,3,144.979408,43,1,3,1,Feminino,Asiático,Com condições de saúde adicionais
4,4,162.502954,55,0,2,1,Masculino,Hispânico,Com condições de saúde adicionais


Verique a base pacientes.csv, os dados são ctícios. A sintetiza uma base
de dados de um hospital que trata de pacientes com problemas cardíacos.
Considere a base como a população, portanto as estatísticas da população
são conhecidas. Responda as perguntas abaixo:
3. Considerando uma amostra de 45 números que representam o index do
dataframe, índices= ([909, 751, 402, 400, 726, 39, 184, 269, 255, 769,
209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967,
699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348,
146, 505, 273, 957, 362]). Considerando essa amostra é possível dizer
que a idade média das pessoas com problemas cardíacos é maior que
50 anos? Nível de signicância igual a 5%.


In [ ]:
# Índices da amostra da questão 3
indices_q3 = [909, 751, 402, 400, 726, 39, 184, 269, 255, 769,
              209, 715, 677, 381, 793, 697, 89, 280, 232, 756,
              358, 36, 439, 768, 967, 699, 473, 222, 89, 639,
              883, 558, 757, 84, 907, 895, 217, 224, 311, 348,
              146, 505, 273, 957, 362]

# Amostra de idades
amostra_idade = df_pacient.loc[indices_q3, "Idade"]

# Parâmetros do teste
mu = 50  # média populacional sob H0
alpha = 0.05

# Teste t unilateral (H1: média > 50)
t_stat, p_value = stats.ttest_1samp(amostra_idade, popmean=mu)
p_value_unilateral = p_value / 2  # cauda à direita

# Verifica se a média da amostra é maior que 50
media_amostra = amostra_idade.mean()
resultado = "Rejeita H0" if (p_value_unilateral < alpha and media_amostra > mu) else "Não rejeita H0"

media_amostra, t_stat, p_value_unilateral, resultado

<ipython-input-44-0217a73d029f>:16: DeprecationWarning: Please import `ttest_1samp` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  t_stat, p_value = stats.ttest_1samp(amostra_idade, popmean=mu)


(np.float64(50.08888888888889),
 np.float64(0.052280983519350496),
 np.float64(0.4792707233816392),
 'Não rejeita H0')

4. Queremos entender que tipo de amostra estamos lidando se dividirmos
os conjuntos em 2, sendo um com pessoas que têm condições de
saúde adicionais e o outro com pessoas saudáveis. Seria dependente
ou independente?

In [ ]:
# Separar os grupos com e sem condições adicionais
grupo_com_condicao = df_pacient[df_pacient["Nome_Estado_Saude"] != "Saudável"]
grupo_saudavel = df_pacient[df_pacient["Nome_Estado_Saude"] == "Saudável"]

# Verificar se os grupos têm interseção nos índices
indices_com = set(grupo_com_condicao.index)
indices_sem = set(grupo_saudavel.index)
intersecao = indices_com & indices_sem

# Verifica dependência com base na interseção de índices
tipo_amostra = "Dependente" if len(intersecao) > 0 else "Independente"
tipo_amostra


'Independente'

5. Agora considere o um conjunto de pessoas aleatória que representam
o index do dataframe, índices = ([690, 894, 67, 201, 364, 19, 60, 319,
588, 643, 855, 623, 530, 174, 105, 693, 6, 462, 973, 607, 811, 346, 354,966, 943, 372]), podemos dizer que a pressão arterial média para
pacientes com condições de saúde adicionais é igual à pressão arterial
média para pacientes sem condições adicionais de saúde? Considere o
nível de signicância a 6%.


In [ ]:
indices_5 = [690, 894, 67, 201, 364, 19, 60, 319, 588, 643, 855, 623, 530, 174,
             105, 693, 6, 462, 973, 607, 811, 346, 354, 966, 943, 372]
amostra_5 = df_pacient.loc[indices_5]
grupo_doente = amostra_5[amostra_5['Estado_Saude'] == 1]['Pressao_Arterial']
grupo_saudavel = amostra_5[amostra_5['Estado_Saude'] == 0]['Pressao_Arterial']
t_stat, p_valor = stats.ttest_ind(grupo_doente, grupo_saudavel)
print(f'5. Teste t entre grupos de saúde:\n- Média doentes: {grupo_doente.mean():.1f}\n- Média saudáveis: {grupo_saudavel.mean():.1f}\n- p-valor: {p_valor:.4f}\n')

5. Teste t entre grupos de saúde:
- Média doentes: 153.2
- Média saudáveis: 152.9
- p-valor: 0.9359



<ipython-input-46-7c17639c3e53>:6: DeprecationWarning: Please import `ttest_ind` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  t_stat, p_valor = stats.ttest_ind(grupo_doente, grupo_saudavel)


6. Existe uma diferença signicativa na pressão arterial média entre
diferentes grupos étnicos nesta população? (Teste ANOVA, alpha é 5%)
a. Hipótese Nula (H0): A pressão arterial média é a mesma em todos
os grupos étnicos.
b. Hipótese Alternativa (H1): Há uma diferença signicativa na
pressão arterial média entre pelo menos dois grupos étnicos
nesta população.


In [ ]:
# ANOVA
grupos = [group['Pressao_Arterial'].values for name, group in df_pacient.groupby('Nome_Etnia')]
f_stat, p_value = stats.f_oneway(*grupos)
p_value


<ipython-input-47-49cc797bf585>:3: DeprecationWarning: Please import `f_oneway` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  f_stat, p_value = stats.f_oneway(*grupos)


np.float64(0.873800123850745)

7. Lúcia é uma pesquisadora e tem o objetivo de entender a relação de
gênero neste grupo de pacientes. Acredita-se que há uma relação
entre o sexo e condições de saúde adicionais. (Teste qui-quadrado)

In [ ]:
tabela = pd.crosstab(df_pacient['Genero'], df_pacient['Estado_Saude'])
chi2, p_valor, _, _ = chi2_contingency(tabela)
print(f'7. Teste qui-quadrado:\n- Estatística χ²: {chi2:.2f}\n- p-valor: {p_valor:.4f}\n')

7. Teste qui-quadrado:
- Estatística χ²: 0.00
- p-valor: 1.0000



8. Existe uma associação entre a idade dos pacientes e sua pressão
arterial?
a. Hipótese Nula (H0): se a pressão arterial é independente da idade
b. Hipótese Alternativa (H1): a pressão arterial esta associada a
idade

In [ ]:
corr, p_value = stats.pearsonr(df_pacient['Idade'], df_pacient['Pressao_Arterial'])
corr, p_value


9. Qual é o intervalo de confiança para a média da pressão arterial entre
os pacientes com condições de saúde adicionais? (nível de conança
95%)

In [ ]:
doentes = df_pacient[df_pacient['Estado_Saude'] == 1]['Pressao_Arterial']
media = doentes.mean()
erro = stats.sem(doentes) * t.ppf(0.975, len(doentes) - 1)
print(f'9. IC 95% para pressão (doentes):\n- ({media - erro:.1f}, {media + erro:.1f}) mmHg\n')

9. IC 95% para pressão (doentes):
- (149.2, 151.0) mmHg



<ipython-input-54-9da31db6964f>:3: DeprecationWarning: Please import `sem` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  erro = stats.sem(doentes) * t.ppf(0.975, len(doentes) - 1)


10. A distribuição da pressão arterial na população segue uma distribuição
normal?
a. Hipótese Nula (H0): A distribuição da pressão arterial na
população segue uma distribuição normal.
b. Hipótese Alternativa (H1): A distribuição da pressão arterial na
população não segue uma distribuição normal.

In [ ]:
stat, p_valor = shapiro(df_pacient['Pressao_Arterial'])
print(f'10. Teste de normalidade (Shapiro-Wilk):\n- Estatística W: {stat:.2f}\n- p-valor: {p_valor:.4f}\n')

10. Teste de normalidade (Shapiro-Wilk):
- Estatística W: 1.00
- p-valor: 0.0607

